## Hamiltonian Simulation Observable Benchmark Tests - WIP

This notebook is being used to flesh out the details of integrating the HamLib utility module, the HamLib kernel function, and the newly developed functions for generating groups of commuting terms.

From any specific Hamiltonian, we create a base evolution circuit with 0 - N Trotter steps, initialize the circuit, and append the necessary basis rotations before measuring and computing the values of energy and other related observables.

At a later point, the execution code and plotting code will also be integrated into the benchmark framework. For now, however, the focus is on developing the code necessary to evaluate the performance of observable execution using various optimization options.


### Setup Dependencies and Initialize Modules

This cell is used to pull in dependencies from standard libraries, the Hamlib helper functions, and the newly created observables module (in WIP).
Initialize the modules so a re-run from the beginning is repeatable.

In [ ]:
import numpy as np
from math import sin, cos, pi
import time

# Import Qiskit and Qiskit Pauli operator classes
from qiskit.quantum_info import Pauli, SparsePauliOp
from qiskit import QuantumCircuit

# Initialize simulator backend
from qiskit_aer import Aer
backend = Aer.get_backend('qasm_simulator')

# Import HamLib helper functions (from _common)
from hamlib._common import hamlib_utils

# Import Observable helper functions
from hamlib._common import observables

# Import Hamlib Simulation kernel (from qiskit)
from hamlib.qiskit import hamlib_simulation_kernel
    

### Configure Options for Execution

In [2]:
# Qubit width of the Hamiltonian
num_qubits = 4

# Parameters of Trotterized simulation
K = 1
t = 0.1

# for debugging Hamlib helper functions
hamlib_simulation_kernel.verbose = False
hamlib_utils.verbose = False

# for debugging observables module
observables.verbose_circuits = False


### Specify a Hamiltonian for Testing
Choose a Hamiltonian from HamLib or create a custom Hamiltonian


In [24]:
# Specify the desired Hamiltonian from HamLib 
# Set the hamiltonian_name: 'TFIM', 'Fermi-Hubbard-1D', 'Bose-Hubbard-1D', 'Heisenberg' or 'Max3Sat'
# If no name, it means we use an explicitly specified Hamiltonian
hamiltonian_name = 'TFIM'

########### HamLib Hamiltonians

if hamiltonian_name == 'TFIM':
    hamiltonian_params = { "1D-grid": "pbc", "h": 2 }

if hamiltonian_name == 'Heisenberg':
    hamiltonian_params = { "1D-grid": "nonpbc", "_h": 1 }
    
if hamiltonian_name == 'Fermi-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "pbc", "enc": "bk", "U":12 }

if hamiltonian_name == 'Bose-Hubbard-1D':
    hamiltonian_params = { "1D-grid": "nonpbc", "enc": "gray", "U":10 }

if hamiltonian_name == 'Max3Sat':
    hamiltonian_params = { "ratio": "2", "rinst": "02" }

if hamiltonian_name == 'chemistry/electronic/standard/H2':
    hamiltonian_params = { "ham_BK": '' }
    
########### Explicit Hamiltonians

# classical simple Ising is ZZ
# TFIM ZZ + X  is transverse field
# + longitudinal field -> ZZ, X, and Z
def get_ising_hamiltonian(L, J, h, alpha=0):

    # List of Hamiltonian terms as 3-tuples containing
    # (1) the Pauli string,
    # (2) the qubit indices corresponding to the Pauli string,
    # (3) the coefficient.
    ZZ_tuples = [("ZZ", [i, i + 1], -J) for i in range(0, L - 1)]
    Z_tuples = [("Z", [i], -h * sin(alpha)) for i in range(0, L)]
    X_tuples = [("X", [i], -h * cos(alpha)) for i in range(0, L)]

    # We create the Hamiltonian as a SparsePauliOp, via the method
    # `from_sparse_list`, and multiply by the interaction term.
    hamiltonian = SparsePauliOp.from_sparse_list([*ZZ_tuples, *Z_tuples, *X_tuples], num_qubits=L)
    return hamiltonian.simplify()

# extract Hamiltonian from HamLib
if hamiltonian_name != '':

    # load the HamLib file for the given hamiltonian name
    hamlib_utils.load_hamlib_file(filename=hamiltonian_name)

    # return a sparse Pauli list of terms queried from the open HamLib file
    sparse_pauli_terms, dataset_name = hamlib_utils.get_hamlib_sparsepaulilist(num_qubits=num_qubits, params=hamiltonian_params)
    print(f"... sparse_pauli_terms = \n{sparse_pauli_terms}")
    """
    sparse_pauli_terms = [#({}, (1+0j)), 
                          ({0: 'X', 1: 'X'}, (1+0j)), ({0: 'Y', 1: 'Y'}, (1+0j)), ({0: 'Z', 1: 'Z'}, (1+0j)),
                          #({0: 'X', 3: 'X'}, (1+0j)), ({0: 'Y', 3: 'Y'}, (1+0j)), ({0: 'Z', 3: 'Z'}, (1+0j)),
                          ({1: 'X', 2: 'X'}, (1+0j)), ({1: 'Y', 2: 'Y'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)),
                          ({3: 'X', 2: 'X'}, (1+0j)), ({3: 'Y', 2: 'Y'}, (1+0j)), ({3: 'Z', 2: 'Z'}, (1+0j)),
                          #({0: 'Z'}, (1+0j)), ({1: 'Z'}, (1+0j)), ({3: 'Z'}, (1+0j)), ({2: 'Z'}, (1+0j))
                         ]
    """
    sparse_pauli_terms = [
                          ({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 3: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({3: 'Z', 2: 'Z'}, (1+0j)), 
                          ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({2: 'X'}, (2+0j))
                         ]
    
    # convert the SparsePauliList to a SparsePauliOp object
    sparse_pauli_op = hamlib_simulation_kernel.ensure_sparse_pauli_op(sparse_pauli_terms, num_qubits)
    print(f"... sparse_pauli_op = \n{sparse_pauli_op}")
    print("")
    
    # convert SparsePauliOp to list form (for use by expectation functions below)
    pauli_terms = sparse_pauli_op.to_list()
    
# create explicit Hamiltonian
else:
    sparse_pauli_op = get_ising_hamiltonian(L=num_qubits, J=0.2, h=1.2, alpha=pi / 8)
    pauli_terms = sparse_pauli_op.to_list()

print(f"... Hamiltonian name = {hamiltonian_name}")


print(pauli_terms)
print("")


... sparse_pauli_terms = 
[({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 3: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({3: 'Z', 2: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({2: 'X'}, (2+0j))]
... sparse_pauli_op = 
SparsePauliOp(['ZZII', 'ZIIZ', 'IZZI', 'IIZZ', 'XIII', 'IXII', 'IIIX', 'IIXI'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 2.+0.j, 2.+0.j, 2.+0.j, 2.+0.j])

... Hamiltonian name = TFIM
[('ZZII', (1+0j)), ('ZIIZ', (1+0j)), ('IZZI', (1+0j)), ('IIZZ', (1+0j)), ('XIII', (2+0j)), ('IXII', (2+0j)), ('IIIX', (2+0j)), ('IIXI', (2+0j))]



### Create a Hamiltonian Simulation Circuit
Initialize the circuit to a known initial state, append 'K' Trotter steps for total time 't', followed by basis rotation gates.

In [25]:
# create the HamLib Simulation kernel, from the current Hamiltonian.
# We use a different method for kernel creation with HamLib than with custom Hamiltonians

########### create Trotterized evolution circuit for HamLib Hamiltonians

# create Hamiltonian evolution circuit from HamLib
if hamiltonian_name != '':
    init_state = "checkerboard"

    qc, bitstring = hamlib_simulation_kernel.HamiltonianSimulation(
        num_qubits=num_qubits, 
        ham_op=sparse_pauli_terms,
        K=K, t=t,
        init_state = init_state,
        append_measurements = False,
        method = 1, 
    )    
    
########### create Trotterized evolution circuit for custom Hamiltonian
else:
    #qc = None
    qc = QuantumCircuit(num_qubits)

print(f"... Trotterized Circuit, K={K}, t={t}")
if num_qubits < 11:
    if hamiltonian_name != '':
        hamlib_simulation_kernel.kernel_draw(qc, method=1)
    else:
        print(qc)
else:
    print("  ... circuit is too large to draw.")

print("")


... Trotterized Circuit, K=1, t=0.1
Sample Circuit:
  H = [({0: 'Z', 1: 'Z'}, (1+0j)), ({0: 'Z', 3: 'Z'}, (1+0j)), ({1: 'Z', 2: 'Z'}, (1+0j)), ({3: 'Z', 2: 'Z'}, (1+0j)), ({0: 'X'}, (2+0j)), ({1: 'X'}, (2+0j)), ({3: 'X'}, (2+0j)), ({2: 'X'}, (2+0j))]
     ┌────────┐ ░ ┌──────────────┐ ░ 
q_0: ┤0       ├─░─┤0             ├─░─
     │        │ ░ │              │ ░ 
q_1: ┤1       ├─░─┤1             ├─░─
     │  Neele │ ░ │  e^-iHt(0.1) │ ░ 
q_2: ┤2       ├─░─┤2             ├─░─
     │        │ ░ │              │ ░ 
q_3: ┤3       ├─░─┤3             ├─░─
     └────────┘ ░ └──────────────┘ ░ 
  Initial State Neele:
     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────
          
  Evolution Operator (e^-iHt) =
                                               ┌─────────┐
q_0: ───────────■─────────────────────■────────┤ Rx(0.4) ├
                │                     │ZZ(0.2) ├─────────┤
q_1: ───────────┼──────────■──────────■────────┤ Rx(0.4) ├
                │

### Compute Energy of the Hamiltonian - Unoptimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [26]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = False

N = 3
print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
        
    # Estimate expectation for the Ham terms using the circuit qc
    total_energy, term_contributions = observables.estimate_expectation_value(backend, qc, pauli_terms, 
                                             use_commuting_groups=use_commuting_groups, num_shots=100000)
    print(f"\nTotal Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")


************ Compute energy of the Hamiltonian 3 times

Total Energy: (-3.41374+0j)
Term Contributions: {'ZZII': -0.85024, 'ZIIZ': -0.84854, 'IZZI': -0.85098, 'IIZZ': -0.84814, 'XIII': 0.0001, 'IXII': -0.00526, 'IIIX': -0.0018, 'IIXI': -0.00096}

Total Energy: (-3.3604000000000003+0j)
Term Contributions: {'ZZII': -0.84708, 'ZIIZ': -0.84792, 'IZZI': -0.84902, 'IIZZ': -0.84922, 'XIII': 0.00318, 'IXII': 0.00454, 'IIIX': 0.00614, 'IIXI': 0.00256}

Total Energy: (-3.39506+0j)
Term Contributions: {'ZZII': -0.85056, 'ZIIZ': -0.8498, 'IZZI': -0.8488, 'IIZZ': -0.8489, 'XIII': 0.00328, 'IXII': -0.00418, 'IIIX': 0.00136, 'IIXI': 0.00104}



### Compute Energy of the Hamiltonian - Optimized

Here, we convert the Hamiltonian to a set of circuits, using commuting groups if specified,
and execute the circuits in order to compute the expectation value 

In [5]:
# Flag to control optimize by use of commuting groups
use_commuting_groups = True

N = 3
print(f"\n************ Compute energy of the Hamiltonian {N} times")

# Calculate the total energy
for i in range(N):
        
    # Estimate expectation for the Ham terms using the circuit qc
    total_energy, term_contributions = observables.estimate_expectation_value(backend, qc, pauli_terms, 
                                             use_commuting_groups=use_commuting_groups, num_shots=10000)
    print(f"\nTotal Energy: {total_energy}")
    print(f"Term Contributions: {term_contributions}")
    
print("")


************ Compute energy of the Hamiltonian 3 times


NameError: name 'qc' is not defined

### Compute expectation value of other observables.
Note that this only works for the Ising model.

In [10]:
# Define additional Hamiltonian terms for other Ising observables 
H_terms_spin_correlation = observables.swap_pauli_list([(0.2,'IIIIZZ'), (0.2,'IIIZZI'), (0.2,'IIZZII'), (0.2,'IZZIII'), (0.2,'ZZIIII')])
H_terms_magnetization = observables.swap_pauli_list([(1,'IIIIIZ'), (1,'IIIIZI'), (1,'IIIZII'), (1,'IIZIII'), (1,'IZIIII'), (1, 'ZIIIII')])

spin_correlation = observables.calculate_expectation_from_contributions(term_contributions, H_terms_spin_correlation)
print(spin_correlation)

magnetization = observables.calculate_expectation_from_contributions(term_contributions, H_terms_magnetization)
print(magnetization)


WARN: term not found in term_contributions: IIIIZZ
WARN: term not found in term_contributions: IIIZZI
WARN: term not found in term_contributions: IIZZII
WARN: term not found in term_contributions: IZZIII
WARN: term not found in term_contributions: ZZIIII
0.0
WARN: term not found in term_contributions: IIIIIZ
WARN: term not found in term_contributions: IIIIZI
WARN: term not found in term_contributions: IIIZII
WARN: term not found in term_contributions: IIZIII
WARN: term not found in term_contributions: IZIIII
WARN: term not found in term_contributions: ZIIIII
0
